In [1]:
import os
import shutil
from tensorflow.python.client import device_lib
import keras
from keras import layers, models, optimizers
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout, Input
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import load_img
from PIL import Image
import os
import shutil

import matplotlib.pyplot as plt

In [ ]:
# Dataset personnes
!git clone https://gitlab.enseeiht.fr/aandre2/Dataset_Market-1501-v15.09.15

# Dataset pokemons
!git clone https://gitlab.enseeiht.fr/nurbani/dataset-pokemon


Cloning into 'Dataset_Market-1501-v15.09.15'...
remote: Counting objects: 77323, done.
remote: Compressing objects: 100% (70371/70371), done.
^C
Cloning into 'dataset-pokemon'...
remote: Counting objects: 26364, done.
remote: Compressing objects: 100% (25090/25090), done.


On commence par les chemins

In [ ]:
path_pokemon = "/content/dataset-pokemon/"

In [ ]:
#Chemin vers les pokemon qu'on va utiliser
pokemon_1G_red_blue = os.listdir(path_pokemon + '1G-red-blue')[:151]
pokemon_1G_red_green = os.listdir(path_pokemon + '1G-red-green')[:151]
pokemon_1G_yellow = os.listdir(path_pokemon + '1G-yellow')[:151]
pokemon_2G_crystal = os.listdir(path_pokemon + '2G-crystal')[:151]
pokemon_2G_gold = os.listdir(path_pokemon + '2G-gold')[:151]
pokemon_2G_silver = os.listdir(path_pokemon + '2G-silver')[:151]
pokemon_3G_emerald = os.listdir(path_pokemon + '3G-emerald')[:151]
pokemon_3G_firered_leafgreen = os.listdir(path_pokemon + '3G-firered-leafgreen')[:151]
pokemon_3G_ruby_sapphire = os.listdir(path_pokemon + '3G-ruby-sapphire')[:151]
pokemon_4G_diamond_pearl = os.listdir(path_pokemon + '4G-diamond-pearl')[:151]
pokemon_4G_heartgold_soulsilver = os.listdir(path_pokemon + '4G-heartgold-soulsilver')[:151]
pokemon_4G_platinum = os.listdir(path_pokemon + '4G-platinum')[:151]
pokemon_5G_black_white = os.listdir(path_pokemon + '5G-black-white')[:151]

Création du fichier de données d'entraînement

In [ ]:
# path train
path_train_pokemon = path_pokemon + "train/"

# train folder
if not os.path.exists(path_train_pokemon):
  os.mkdir(path_train_pokemon)

# train images
for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "1G-red-blue/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-1G-red-blue.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "1G-red-green/" + str(filename) +'.png', path_train_pokemon + str(filename) + '-1G-red-green.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "1G-yellow/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-1G-yellow.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "2G-crystal/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-2G-crystal.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "2G-gold/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-2G-gold.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "2G-silver/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-2G-silver.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "3G-emerald/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-3G-emerald.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "3G-firered-leafgreen/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-3G-firered-leafgreen.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "3G-ruby-sapphire/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-3G-ruby-sapphire.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "4G-diamond-pearl/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-4G-diamond-perl.png')

for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "4G-heartgold-soulsilver/" + str(filename) + '.png', path_train_pokemon + str(filename) + '-4G-heartgold-soulsilver.png')

# train labels
train_filenames = os.listdir(path_train_pokemon)
train_categories=[]
for filename in train_filenames:
    train_categories.append(filename.split('-')[0])

Création du fichier des données de validation

In [ ]:
# path validation
path_validation_pokemon = path_pokemon + "validation/"

# validation folder
if not os.path.exists(path_validation_pokemon):
  os.mkdir(path_validation_pokemon)


# Creation
if not os.path.exists(path_validation_pokemon):
  os.mkdir(path_validation_pokemon)

# Validation images
for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "5G-black-white/" + str(filename) + '.png', path_validation_pokemon + str(filename) + '-5G-black-white.png')

# validation labels
validation_filenames = os.listdir(path_validation_pokemon)
validation_categories=[]
for filename in validation_filenames:
  validation_categories.append(filename.split('-')[0])

Création du fichier des données de test

In [ ]:
# path test
path_test_pokemon = path_pokemon + "test/"

# testfolder
if not os.path.exists(path_test_pokemon):
  os.mkdir(path_test_pokemon)


# Test images
for filename in range(1, 152):
  shutil.copyfile(path_pokemon + "4G-platinum/" + str(filename) + '.png', path_test_pokemon + str(filename) + '-4G-platinum.png')

# Test labels
test_filenames = os.listdir(path_test_pokemon)
test_categories=[]
for filename in test_filenames:
  test_categories.append(filename.split('-')[0])

Création des ensemble d'apprentissage, test et validation

In [ ]:
#Images d'entrainement
train_df = pd.DataFrame({
    'filename': train_filenames,
    'category': train_categories
})


#Images de validation
validation_df = pd.DataFrame({
    'filename': validation_filenames,
    'category': validation_categories
})


#Images de test
test_df = pd.DataFrame({
    'filename': test_filenames,
    'category': test_categories
})


train_df['category'] = train_df['category'].astype(str)
validation_df['category'] = validation_df['category'].astype(str)
test_df['category'] = validation_df['category'].astype(str)

total_train = train_df.shape[0]
total_validate = validation_df.shape[0]

In [ ]:
import random
from keras.preprocessing.image import load_img

sample = random.choice(test_filenames)
image = load_img(path_test_pokemon + sample)
plt.imshow(image)
plt.show()

Ensemble d'entraînement, test et validation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 16
image_size = 32


#Images d'entrainement
train_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    path_train_pokemon,
    x_col='filename',
    y_col='category',
    target_size=(image_size,image_size),
    batch_size=batch_size
)

#Images de validation
validation_datagen=ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validation_df, 
    path_validation_pokemon, 
    x_col='filename',
    y_col='category',
    target_size=(image_size, image_size),
    batch_size=batch_size
)

#Images de test
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    path_test_pokemon, 
    x_col='filename',
    y_col='category',
    target_size=(image_size, image_size),
    batch_size=batch_size
)

Entraînement du réseau de neurone.

On utilise un VGG pré-entraîné sur imagenet



Import VGG

In [ ]:
#VGG pré entraîné sur imagenet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

#Mon input c'est du 40 40 3
input = Input(shape=(32, 32, 3),name = 'image_input')

output_vgg16_conv = model_vgg16_conv(input)

#On rajoute quelques layers
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(151, activation='softmax', name='predictions')(x)

#Mon modele à moi
my_model = Model(inputs=input, outputs=x)

my_model.summary()

C'est parti pour l'entraînnement !

In [ ]:
my_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=3e-4),
              metrics=['acc'])

In [ ]:
print("Train_data shape", np.shape(Train_data))
print("Train labels shape", np.shape(Train_labels))

In [ ]:
history = my_model.fit(train_generator,
                    steps_per_epoch=total_train//batch_size,
                    validation_data=validation_generator,
                    validation_steps=total_validate//batch_size,
                    epochs=30,
                    )